# SFDA Survey (Source-free Domain Adaptation)

## 1. Introduction

### UDA와 SFDA의 차이

<img src="./Fig1.png" alt="nn" width="600">

- UDA: Source data, Target data, Source label에 접근 가능
- SFDA: 오직 Target data에만 접근 가능 (Source에는 접근 불가)
    - 정보 보호 및 저작권 문제로부터 자유로움
    - 리소스가 제한된 장치에서도 학습 가능
- MEMO: SFDA 연구는 여러 가지 기술적 접근 방식을 포함하고 있으며, 이들은 대체로 데이터 기반(data-based) 방법과 모델 기반(model-based) 방법으로 분류될 수 있습니다. 데이터 기반 방법은 주로 타겟 데이터의 재구성 및 증강을 통해 도메인 차이를 줄이는 반면, 모델 기반 방법은 주로 소스 모델의 지식을 유지하면서 타겟 도메인에 적응하는 방식을 사용합니다. 이러한 방법들 각각은 도메인 적응의 문제를 해결하는 데 있어 중요한 기여를 합니다.

### SFDA의 분류

<img src="./Fig2.png" alt="nn" width="800">

<img src="./Table1.png" alt="nn" width="800">

## 2. Overview

### Notation

<img src="./Table2.png" alt="nn" width="400">

### SFDA의 작업

- 소스 도메인 $\mathcal{D}^{s} = \{ \{ \mathcal{X}^s, \mathcal{P}(\mathcal{X}^s), d^s \}, \mathbf{L}^s \}$에서 학습된 모델 $\mathcal{M}$이 주어진다.
- Pre-training stage: 소스 모델은 지도학습으로 pre-train된다.
- Adaptation stage: 소스 도메인 데이터는 접근 불가능하며, SFDA의 목적은 Source-trained 모델이 Target domain $\Phi^t = \{\{\mathcal{X}^t, \mathcal{P}(\mathcal{X}^t), d^t \}\}$에 적응하도록 하는 것이다.
    - 이때, $\mathcal{M}(\mathcal{X}^t) = \mathcal{P}(\mathcal{Y}^t|\mathcal{X}^t)$ 의 모델의 출력이 높은 정확도를 갖도록 하는 것을 목표로 한다.
    - $\mathcal{P}(\mathcal{Y}^t|\mathcal{X}^t)$는 target data $\mathcal{X}^t$에 대한 예측 결과이다.

### SFDA의 setting
```대부분의 SFDA는 Closed-set, Single-Source 셋팅에 초점을 맞춘다```

- 특징 공간의 동일성 여부
    - 이질 도메인 적응(Heterogeneous DA): $\mathcal{P}(\mathcal{X}^s) \neq \mathcal{P}(\mathcal{X}^t), d_s \neq d_t$인 상태로, 분포 적응과 특징 공간 적응이 모두 필요한 상태
    - 동질 도메인 적응(Homogeneous DA): $\mathcal{P}(\mathcal{X}^s) \neq \mathcal{P}(\mathcal{X}^t), d_s = d_t$인 상태로 특징 공간이 동일한 상태로 분포 적응만 수행 (대부분의 SFDA)
- 레이블 집합의 대응관계에 따른 분류
    - Closed-set
    - Partial-set
    - Open-Set
- 소스와 타겟 도메인 수에 따른 분류
    - Single-Source
    - Multi-Source
    - Multi-Task

<img src="./Fig3.png" alt="nn" width="600">

## 3. Data-Based Method

- Domain-based Reconstruction: 모델에 내재된 소스 도메인 정보를 통해 소스 도메인 데이터를 모방하거나, 중간 도메인 혹은 다른 도메인을 재구성
- Image-based information extraction: 레이블이 없는 타겟 도메인 데이터에서 잠재적인 데이터 구조나 클러스터링 정보를 탐색하여 타겟 데이터만으로 도메인 적응 작업을 수행

### Domain-based Reconstruction

#### [ Virtual Domain Generation ]

$$
\mathcal{L}_{UDA_M} = \mathcal{L}_{cls}(\mathcal{C}(\mathcal{F}(\mathcal{X}^s)), \mathcal{Y}^s)+\mathcal{L}_{div}(\mathcal{F}(\mathcal{X}^s), \mathcal{F}(\mathcal{X}^t)) \tag{1}
$$

- 이 함수는 UDA에서 모델의 소스 도메인에서 지도 학습을 수행함과 동시에 타겟 도메인의 분포를 직접 정렬하는 일반적 패러다임이다.
- $\mathcal{L}_{cls}$: 소스 도메인에서의 지도 학습 손실로, 주로 cross-entropy를 사용
- $\mathcal{L}_{div}$: 정렬에 사용되는 기준함수
- 다만, SFDA는 $\mathcal{X}^s$를 adaptation에 사용할 수 없다.
- 따라서 VDG에서는 source data의 부재를 보충하기 위한 새로운 도메인을 구성한다.

$$
\mathcal{L}_{SFDA} = \mathcal{L}_{gen}(\mathcal{C}(\mathcal{F}(\mathcal{X}^v)))+\mathcal{L}_{div}(\mathcal{F}(\mathcal{X}^v), \mathcal{F}(\mathcal{X}^t)) \tag{2}
$$

- $\mathcal{X}_v$: 가상 도메인을 의미한다.
- $\mathcal{L}_{gen}$: 가상 도메인의 생성 손실을 나타낸다.
- 모델은 소스 도메인에서 사전 학습되었고, 따라서 모델의 출력을 탐색하여 도메인 분포를 시뮬레이션한다.
- 식 3과 비교하였을 때, SFDA에서는 도메인 간의 정렬뿐만 아니라 가상 도메인 생성의 품질도 고려해야 한다.
- 가상 도메인이 생성되면 SFDA의 정렬 문제는 UDA에서의 정렬 문제와 유사해진다.
- ex:
    - ADDA가 VDA-DA에서 사용된다.
    - MMD가 STDA에서 사용된다.
- 가상 도메인은 adversarial generarion 혹은 Gaussian distribution에 기반하여 생성될 수 있다.

$$
\mathcal{L(C, F, D,} G) = \mathcal{L}_{adv}+\mathcal{L}_{con} \tag{3}
$$
    
- GAN(Generative Adversarial Network)의 아이디어에서 나온 adversarial generation이다.
- $\mathcal{L}_{con}$: 도메인 간의 의미적 일관성을 유지하기 위한 손실함수이다.
- $\mathcal{L}_{adv}$: domain discriminator 간의 적대적 손실을 나타낸다.
    - 이는 (4)의 방정식을 따라 유도된다.

$$
\mathcal{L}_{adv}(G) = \mathbb{E}_{y,\tilde{x}}[\log \mathcal{D}(1-G(y, \tilde{z}))], \\
\mathcal{L}_{adv}(\mathcal{D}) = \mathbb{E}_{x_t \verb|~| \mathcal{X}_t}[\log \mathcal{D}(x_t)]+\mathbb{E}_{y, \tilde{z}}[\log (1-\mathcal{D}(G(y, \tilde{z})))] \tag{4}
$$

- $\tilde{z}$: 노이즈 벡터를 나타낸다.
- $G(y, \tilde{z})$: 사전 정의된 레이블 $y$를 조건으로 하는 generator를 나타낸다.
- 3C-GAN: generator에 의해 생성된 $x_v=G(y, \tilde{z})$와 레이블 $y$ 사이의 의미적 유사성 강조
- classifier: 성능 향상을 위해 deterministic constraint, weight constraint, clustering-based contraint 제안.
- SDDA: domain discriminator 기반의 consistency loss 제안.
- CPGA: contrastive loss 추가

$$
\tilde{x}=G(\tilde{z}),~~\tilde{z} \verb|~| \mathcal{N}(0, 1) \tag{5}
$$
    
- VDG를 위해 Gaussian distribution을 기반으로 소스 도메인의 분포를 추정

$$
D^v(f_v)=\sum_{k=1}^{K} \pi_k\mathcal{N}(f_v|\mu_k, \sigma^2I) \tag{6}
$$

- 소스 도메인의 분포를 여러 가우시안 분포의 혼합으로 간주.
- VDM-DA: 가상 도메인의 분포를 특징 공간에서 위의 가우시안 혼합 모델로 구성
- $f_v$: 가상 특징
- $\pi_k$: 혼합 계수이며 $\sum_{k=1}^{K} \pi_k=1$이다.
- 매개변수 추정을 위해 경험적으로 $K$를 범주의 수로 설정
- 소스 분류기의 가중치가 암묵적으로 각 범주에 대한 prototypical information을 포함한다고 간주
- 모델의 평균과 표준 편차를 소스 분류기에 기반하여 도출

$$
\tilde{x}_{s, aug}=\lambda\tilde{x}_{s}^{i}+(1-\lambda)\tilde{x}_{s}^{j}, \\ \tilde{y}_{s, aug}=\lambda\tilde{y}_{s}^{i}+(1-\lambda)\tilde{y}_{s}^{j} \tag{7}
$$

- 가상 도메인을 위해 생성 모델을 사용하는 것은 비용이 많이 들 뿐만 아니라, 기본 데이터 패턴이 복잡할 때 도메인 일반화(domain generalization)를 잘 수행하기도 어렵다.
- Non-generative 접근 시도: 타겟 도메인에서 신뢰할 수 있는 데이터를 직접 선택하여 가상 소스 도메인을 구성
- 타겟 이미지를 소스 모델에 입력하여 예측 엔트로피가 높은 샘플을 소스 도메인 분포에 더 가까운 것으로 간주
- 그러나 이 방법은 가상 소스 도메인에 대한 데이터 부족 문제를 갖는다.
    - Du: 가상 소스 도메인을 구성하기 위해 선택할 수 있는 샘플의 수가 타겟 도메인의 $1\over{10}$에 불과하다는 것을 발견
- 이러한 문제를 해결하기 위해 Mixup을 이용하여 가상 소스 도메인 데이터를 확장
- $\lambda$: mixup계수
- $\tilde{x}_{s}^{i}$, $\tilde{x}_{s}^{j}$: 가상 소스 도메인에서 예측 엔트로피에 의해 선택된 샘플
- $\tilde{y}_{s}^{i}$, $\tilde{y}_{s}^{j}$: 해당 레이블
- 증강된 소스 도메인: $D_{aug}^{s} = D^s\cup \tilde{D}^{aug}$, $\tilde{D}^{aug}=\{\tilde{x}_{s, aug},\tilde{y}_{s, aug}\}$

***

#### [ Intra-domain Adversarial Exploration ]

**Step 1** Source training:
$$\min_{\theta_{\mathcal{F}}, \theta_{c_1}, \theta_{c_2}} \sum_{i=1}^{2} \mathcal{L}_{\text{cls}}\left( C_i\left( \mathcal{F}\left( \mathcal{X}^s \right) \right), \mathcal{Y}^s \right)$$

**Step 2** Maximum classifier prediction discrepancy:
$$\min_{\theta_{\mathcal{C}_1}, \theta_{\mathcal{C}_2}}\mathcal{L}_{cls}(\mathcal{X}^s, \mathcal{Y}^s)-\mathcal{L}_{dis}(\mathcal{Y}_{1}^{t}|\mathcal{X}^t, \mathcal{Y}_{2}^{t}|\mathcal{X}^t)$$

**Step 3** Minimize classifier prediction discrepancy:
$$\min_{\theta_{\mathcal{F}}}\mathcal{L}_{dis}(\mathcal{Y}_{1}^{t}|\mathcal{X}^t, \mathcal{Y}_{2}^{t}|\mathcal{X}^t)$$

$$
\min_{\theta_{\mathcal{c}_t}} \sum_{x \in \mathcal{X}_h^t} \mathcal{L}_{\text{dis}}\left( p^s(x), p^t(x) \right) - \sum_{x \in \mathcal{X}_l^t} \mathcal{L}_{\text{dis}}\left( p^s(x), p^t(x) \right) \tag{8}
$$

$$
\min_{\theta_{\mathcal{F}}} \sum_{x \in \mathcal{X}^t} \mathcal{L}_{\text{dec}}\left( p^s(x), p^t(x) \right) \tag{9}
$$


$$
\min_{\theta_{\mathcal{C}_t}}-\sum_{i=1}^{n_t}\left(\alpha_{i}^{s} \log \left(\sum_{k=1}^{K}p_{(i)k}^{st}\right) + \alpha_{i}^{t} \log \left(\sum_{k=K+1}^{2K}p_{(i)k}^{st}\right)\right), \\
\min_{\theta_{\mathcal{F}}}-\sum_{i=1}^{n_t}\left(\alpha_{i}^{t} \log \left(\sum_{k=1}^{K}p_{(i)k}^{st}\right) + \alpha_{i}^{s} \log \left(\sum_{k=K+1}^{2K}p_{(i)k}^{st}\right)\right) \tag{10}
$$

$$
\min_{\theta_{\mathcal{C}_t}}\sum_{x\in{\mathcal{X}_{h}^{t}}}\mathcal{D}_{SKL}\left(p^s(x),p^t(x)\right)-\sum_{x\in{\mathcal{X}_{l}^{t}}}\mathcal{D}_{SKL}\left(p^s(x),p^t(x)\right), \\
\min_{\theta_{\mathcal{F}}}\sum_{i=1}^{n_t}\sum_{k=1}^{K}[-p_{i,k}^{s}\log p_{i,k}^{t}-p_{i,k}^{t}\log p_{i,k}^{s}] \tag{11}
$$

$$
\begin{cases}
\overrightarrow{\varepsilon}\left( \mathcal{D}^t, \mathcal{D}^I \right)_{UDA} &= \overrightarrow{\varepsilon}(\mathcal{D}^t, \mathcal{D}^s) + \overrightarrow{\varepsilon}(\mathcal{D}^s, \mathcal{D}^I) \\
\overrightarrow{\varepsilon}\left( \mathcal{D}^t, \mathcal{D}^I \right)_{SFDA} &= \overrightarrow{\varepsilon}(\mathcal{D}^{t+}, \mathcal{D}^t)
\end{cases} \tag{12}
$$

$$
\tilde{x}_{t} = x_t + \frac{G(x_t)}{\|G(x_t)\|_2}\epsilon, \\
\max_{\theta_G}\frac{1}{n_t}\sum_{i=1}^{n_t}\mathcal{L}_{cls}\left(\mathcal{C}(\mathcal{F}(\tilde{x}_t)), \hat{y}_{t}^{i}\right) \tag{13}
$$

$$
\max_{\theta_{\mathcal{F}}, \theta_{\mathcal{C}}}\frac{1}{n_t}\sum_{i=1}^{n_t}\mathcal{L}_{cls}\left(\mathcal{C}(\mathcal{F}(\tilde{x}_t)), \hat{y}_{t}^{i}\right) \tag{14}
$$

$$
\mathcal{L}_{LSC} = -\frac{1}{n} \sum_{i=1}^{n} \sum_{k=1}^{K} \log \left[ p(x_i) \cdot \mathbb{B}_S \left( \mathcal{N}_k \right) \right] + \sum_{c=1}^{C} \text{KL} \left( \overline{p}_c \parallel q_c \right), \\
\mathcal{N}_{\{1, \ldots, K\}} = \left\{ \mathcal{B}_F^j \mid top - K \left( \cos \left( \mathcal{F} \left( x_i \right), \mathcal{B}_F^j \right) \right), \forall \mathcal{B}_F^j \in \mathcal{B}_F \right\}, \\
\overline{p} = \frac{1}{n} \sum_{i=1}^{n} p_c \left( x_i \right), \text{ and } \text{q}_{\{c=1, \ldots, C\}} = \frac{1}{C} \tag{15}
$$


$$
\mathcal{L}_{\mathcal{N}} = -\frac{1}{n_t}\sum_{i}\sum_{k\in\mathcal{N}_{K}^{i}}A_{ik}\mathcal{B}_{S,k}^{T}p_i, \\
\mathcal{L}_{E} = -\frac{1}{n_t}\sum_{i}\sum_{k\in\mathcal{N}_{K}^{i}}\sum_{m\in E_{M}^{k}}r\mathcal{B}_{S, m}^{T}p_i \tag{16}
$$

$$
P\left( \mathcal{C}_i \right) = \prod_{j \in \mathcal{C}_i} p_{ij} = \prod_{j \in \mathcal{C}_i} \frac{e^{p_i^T p_j}}{\sum_{k=1}^{N_t} e^{p_i^T p_k}}, \\
P\left( \mathbb{B}_i \right) = \prod_{j \in \mathbb{B}_i} p_{ij} = \prod_{j \in \mathbb{B}_i} \frac{e^{p_i^T p_j}}{\sum_{k=1}^{N_t} e^{p_i^T p_k}} \tag{17}
$$

$$
\mathcal{L}_{\text{content}} = \left\| \mathcal{F}^N \left( \tilde{x}_t \right) - \mathcal{F}^N \left( x_t \right) \right\|_2, \\
\mathcal{L}_{\text{style}} = \frac{1}{N} \sum_{n=1}^{N} \left\| \mu_{\text{current}}^n - \mu_{\text{stored}}^n \right\|_2 + \left\| \sigma_{\text{current}}^n - \sigma_{\text{stored}}^n \right\|_2 \tag{18}
$$

$$
\mathcal{L}_{intra} = \sigma \left(\tilde{F}_{i, j}\right) \left(\frac{F_{i, j}-\mu \left(F_{i, j}\right)}{\sigma \left(F_{i, j}\right)}\right) + \mu \left(\tilde{F}_{i, j}\right), \\
\mathcal{L}_{inter} = \sigma \left(\tilde{F}_{k, i, j}\right) \left(\frac{F_{k, i, j}-\mu \left(F_{k, i, j}\right)}{\sigma \left(F_{k, i, j}\right)}\right) + \mu \left(\tilde{F}_{k, i, j}\right) \tag{19}
$$

$$
F = \begin{bmatrix}
F_{1,1} & \cdots & F_{1,n_w} \\
\vdots & \ddots & \vdots \\
F_{n_h,1} & \cdots & F_{n_h,n_w}
\end{bmatrix}
$$

$$
c_k = \frac{\sum_{x_t \in \mathcal{X}_t} \delta_k \left( \mathcal{F} \circ \mathcal{G}_t (x_t) \right) \mathcal{G}_t (x_t)}
{\sum_{x_t \in \mathcal{X}_t} \delta_k \left( \mathcal{F}_t (x_t) \right)} \tag{20}
$$

$$
y_{\tilde{t}}=\text{arg}\min_{k}\mathcal{L}_{pse}(\mathcal{G}(x_t), p_k) \tag{21}
$$

$$
y_{\tilde{t}} = \text{arg}\min_{k}\mathcal{F}\circ\mathcal{G}(x_t) \tag{22}
$$

$$
\mathcal{L}_{ent} = -\mathbb{E}_{x_t\in\mathcal{X}_t}\sum_{k=1}^{K}\delta_k(p_t(x_t))\log\delta_k(p_t(x_t)) \tag{23}
$$

$$
\mathcal{L}_{div} = \sum_{k=1}^{K}p_k\log p_k \tag{24}
$$

$$
\mathcal{L}_{IM}=\mathcal{L}_{ent}+\mathcal{L}_{div} \tag{25}
$$

$$
\mathcal{L}_{Info} = -\sum_{v^{+}\in V^{+}}\log \frac{\text{exp}(u^{T}v^{+}/\tau)}{\text{exp}(u^{T}v^{+}/\tau)+\sum_{v^{-}\in V^{-}}\text{exp}(u^{T}v^{-}/\tau)} \tag{26}
$$

$$
\mathcal{L}^{1}_{\text{cdc}} = -\sum_{m=1}^{M} \mathbb{I}_{\hat{y}_u = m} \log \frac{\exp\left(u^T w_s^m / \tau\right)}{\sum_{j=1}^{M} \exp\left(u^T w_s^j / \tau\right)} \tag{27}
$$


$$
\mathcal{L}_{cdc}^{2} = -\log \frac{\text{exp}(s_{ij})}{\sum_{v=1}^{b}\mathbb{I}_{v\neq i}|\gamma_{iv}|\text{exp}(s_{iv})} \tag{28}
$$

$$
\mathcal{L}_{cdc}^{3} = -\sum_{x_q\in X_t}\log \frac{\text{exp}(q^{T}k_{+}^{t-m}/\tau)\gamma_{+}^{t-m}}{\sum_{i=0}^{N}\text{exp}(q^{t}k_{i}^{t-m}/\tau)\gamma_{i}^{t-m}} \tag{29}
$$

$$
\mathcal{L}_{Attn_{self}}(Q, K, V) = softmax\left(\frac{QK^{T}}{\sqrt{d_k}}\right)V \tag{30}
$$

$$
\mathcal{L}_{Attn_{cross}}(Q_s, K_t, V_t) = softmax\left(\frac{Q_{s}K_{t}^{T}}{\sqrt{d_k}}\right)V_t \tag{31}
$$